In [1]:
import pandas as pd

df = pd.read_csv('data/company_ceo_31-03.csv')
df

,companyLabel,ceoLabel
0,Nvidia,Jensen Huang
1,BNP Paribas,Jean-Laurent Bonnafé
2,"Block, Inc.",Jack Dorsey
3,Reliance Industries Limited,Mukesh Ambani
4,Frasers Group,Mike Ashley
...,...,...
340,Banca Transilvania,Ömer Tetik
341,Miniso,Ye Guofu
342,Deutsche Post AG,Tobias Meyer
343,Intel,Michelle Johnston Holthaus


In [1]:
from autovu import answer_no_web

answer_no_web("Who is the current governor of the bank of England?")

/root/autovu/autovu.py:33: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(
/root/autovu/autovu.py:52: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use invoke instead.
  response = llm(messages)


'Answer: Andrew Bailey, Confidence: high.'

In [ ]:
# df['rankingDate'] = pd.to_datetime(df['rankingDate']).dt.date
# df_intermediate = df.groupby(['ranking', 'playerLabel'])['rankingDate'].agg('last').reset_index()
# df_intermediate.sort_values(by=['ranking', 'rankingDate'], ascending=[True, False]).drop_duplicates(subset=['ranking'], keep='first').to_csv('query_5_result.csv', index=False)

In [4]:
from dotenv import load_dotenv
import os

load_dotenv()

serp_key = os.getenv('SERPER_API_KEY')
uni_api_key = os.getenv('UNI_API_KEY')

In [5]:
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    openai_api_base="https://llms-inference.innkube.fim.uni-passau.de",
    api_key=uni_api_key,
    model = "deepseekr1",
    temperature=0.6
)



/tmp/ipykernel_2284/2943525429.py:3: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.utilities import GoogleSerperAPIWrapper


# Initialize search tool
search = GoogleSerperAPIWrapper()

In [ ]:
def generate_answer(query):
    
    custom_prompt = PromptTemplate.from_template(
        "You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided."
        "\nNote that search results are more up-to-date and reliable than the information you have as we are in 2025."
        "\nAnswer briefly and directly, return only the answer label."
        "\nReturn a confidence score based on the provided context: high, medium or low."
        "\n The answer should be in the following format: Answer: your_answer Confidence: your_confidence_score"
        "\nUser Question: {input}"
        "\nSearch results: {search}"
    )

    # Create an LLMChain instead of an agent
    llm_chain = LLMChain(
        llm=llm,
        prompt=custom_prompt,
        verbose=True
    )
    # Get the latest factual information first
    search_result = search.results(query)
        
    formatted_results = "\n".join(
        f"Title: {res['title']}\nSnippet: {res['snippet']}\nDate: {res.get('date', 'N/A')}\n"
        for res in search_result['organic']
    )
    # Pass the query to the agent
    try:
        result = llm_chain.run({"input": query, "search": formatted_results})
        print(f"\nFinal Result: {result.strip()}")
    except Exception as e:
        print(f"An error occurred: {e}")
        
    return result.strip()
        

In [22]:
from tqdm import tqdm
tqdm.pandas()

In [25]:
df['prediction']=df['companyLabel'].progress_apply(lambda x: generate_answer(f"Who is the current CEO of {x}?"))

  0%|          | 0/345 [00:00<?, ?it/s]Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided.
Note that search results are more up-to-date and reliable than the information you have as we are in 2025.
Answer briefly and directly, return only the answer label.
Return a confidence score based on the provided context: high, medium or low.
User Question: Who is the current CEO of Nvidia?
Search results: Title: Jensen Huang - Wikipedia
Snippet: Jensen Huang ; Businessman; electrical engineer; philanthropist · Co-founding Nvidia · President and CEO of Nvidia Corporation (1993–present) · Lori Huang. ​. ( m.
Date: N/A

Title: Jensen Huang - NVIDIA - LinkedIn
Snippet: In 1993, I founded NVIDIA with Chris Malachowsky and Curtis Priem to solve the problem of… · Experience: NVIDIA · Education: Stanford University · Location: ...
Date: N/A

Title: Exec Bios - NVIDIA Newsroom
Snippet: Jensen Huang. Founder, President and CEO. Jensen Huang founded

  1%|          | 2/345 [00:24<1:09:33, 12.17s/it]


> Finished chain.

Final Result: <think>
Okay, so I need to figure out who the current CEO of Nvidia is. I remember hearing that Jensen Huang founded the company, but I'm not entirely sure if he's still the CEO now in 2025. Let me go through the search results provided.

Looking at the first result from Wikipedia, it says Jensen Huang has been the President and CEO since 1993, which is a long time. That gives some confidence that he's still in the role unless there's recent news of a change. The LinkedIn profile also mentions him as the co-founder and doesn't indicate any departure.

The NVIDIA Newsroom explicitly states he's the Founder, President, and CEO, serving since 1993. That seems pretty official. Forbes corroborates this information too, noting that he's been CEO since the company started. 

There are some recent dates in the snippets as well. A YouTube video from January 27, 2025, talks about his vision for the future, which suggests he was active as CEO at least up until th

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


...


> Finished chain.

Final Result: <think>
Alright, let's figure out who the current CEO of 21st Century Fox is. I'm a bit confused because the search results have different information. 

First, I see that Lachlan Murdoch is the Executive Chair and CEO of Fox Corporation in some snippets. But wait, isn't Fox Corporation different from 21st Century Fox? I remember that Disney acquired most of 21st Century Fox, so maybe there's a spin-off or rebranding involved here.

Looking at another result, James Murdoch was the CEO of 21st Century Fox before, but he left. There's a snippet saying he became CEO starting July 1, 2015, but that might be outdated because it's from Jun 16, 2015. 

Another search result mentions that Lachlan Murdoch is the co-chairman of 21st Century Fox, along with his brother James. But now, in Sep 2023, he's the CEO of Fox Corporation. I think after Disney bought 21st Century Fox, the remaining part became Fox Corporation, so maybe Lachlan is leading that.

Wait, but s

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided.
Note that search results are more up-to-date and reliable than the information you have as we are in 2025.
Answer briefly and directly, return only the answer label.
Return a confidence score based on the provided context: high, medium or low.
User Question: Who is the current CEO of Sears?
Search results: Title: Eddie Lampert - Wikipedia
Snippet: Edward Scott Lampert (born July 19, 1962) is an American billionaire businessman. He is the former chief executive and chairman of Sears Holdings, the founder ...
Date: N/A

Title: Sears Executive Team - Comparably
Snippet: Eddie Lampert serves as the CEO / President of Sears. ... Mr. Michael A. Miles was a Special Limited Partner at Forstmann Little & Co. Mr. Miles served as the ...
Date: N/A

Title: Sears - Wikipedia
Snippet: In May 2019, former Sears Holdings chairman and CEO Eddie Lampert, mon

  4%|▍         | 15/345 [06:21<2:19:44, 25.41s/it]


KeyboardInterrupt: 

In [16]:

custom_prompt = PromptTemplate.from_template(
    "You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided."
    "\nNote that search results are more up-to-date and reliable than the information you have as we are in 2025."
    "\nAnswer briefly and directly, return only the answer label."
    "\nReturn a confidence score based on the provided context: high, medium or low."
    "\nUser Question: {input}"
    "\nSearch results: {search}"
)

# Create an LLMChain instead of an agent
llm_chain = LLMChain(
    llm=llm,
    prompt=custom_prompt,
    verbose=True
)

# Query
#query = "Who is the current president/ leader of the organization: All European Academies?"
query = "Who is the current CEO of the company: Intel?"

# Get the latest factual information first
search_result = search.results(query)

# if search_result:
#     print(f"Raw Search API Response: {search_result}\n")
# else:
#     print("Error: Web Search API returned None or an invalid response.")
    
formatted_results = "\n".join(
    f"Title: {res['title']}\nSnippet: {res['snippet']}\nDate: {res.get('date', 'N/A')}\n"
    for res in search_result['organic']
)

# Pass the query to the agent
try:
    result = llm_chain.run({"input": query, "search": formatted_results})
    print(f"\nFinal Result: {result.strip()}")
except Exception as e:
    print(f"An error occurred: {e}")



Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:
You are an AI assistant. Your task is to answer the question directly using the latest information which will be provided.
Note that search results are more up-to-date and reliable than the information you have as we are in 2025.
Answer briefly and directly, return only the answer label.
Return a confidence score based on the provided context: high, medium or low.
User Question: Who is the current CEO of the company: Intel?
Search results: Title: Intel Appoints Lip-Bu Tan as Chief Executive Officer - Newsroom
Snippet: Lip-Bu Tan was appointed chief executive officer of Intel Corporation in March 2025. He also serves on the company's board of directors.
Date: Mar 12, 2025

Title: Intel's new CEO tells customers 'be brutally honest with us' | Reuters
Snippet: In his first remarks as Intel's new CEO, Lip-Bu Tan on Monday outlined a leaner version of the iconic American chipmaker in which he would ...
Date: 15 hours ago

Title: Intel Appoints Lip-Bu Tan as Chief Ex

In [7]:
# Query

#query = "Who is the current president/ leader of the organization: All European Academies?"
#query = "All european Academies current President/ Leader?"
query = "Who is the current CEO of the company: Reliance Industries Limited	"
print(f"Query: {query}\n")

# Get the latest factual information first
search_result = search.results(query)
search_result

Query: Who is the current CEO of the company: Reliance Industries Limited	



{'searchParameters': {'q': 'Who is the current CEO of the company: Reliance Industries Limited\t',
  'gl': 'us',
  'hl': 'en',
  'type': 'search',
  'num': 10,
  'engine': 'google'},
 'answerBox': {'snippet': 'Mukesh Dhirubhai Ambani (born 19 April 1957) is an Indian billionaire and businessman who is the chairman and managing director of Reliance Industries.',
  'snippetHighlighted': ['Mukesh Dhirubhai Ambani'],
  'title': 'Mukesh Ambani - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Mukesh_Ambani'},
 'organic': [{'title': 'Reliance Industries Limited Chairman & Managing Director',
   'link': 'https://www.ril.com/about/chairman-managing-director',
   'snippet': 'RIL Chairman and Managing Director, Mukesh D. Ambani, is a chemical engineer from the Institute of Chemical Technology, Mumbai (erstwhile the University ...',
   'position': 1},
  {'title': 'Reliance Industries Limited Board of Directors - Nita Ambani',
   'link': 'https://www.ril.com/about/board-of-directors',
   'sni

In [17]:
import pandas as pd
resultant_dataframe = pd.read_csv("company_ceo_results.csv")

In [18]:
resultant_dataframe['confidence'] = resultant_dataframe['prediction'].str.extract(r'(?i)\b(high|medium|low)\b')

In [19]:
import re

def extract_explanation(text):
    match = re.search(r'<think>(.*?)</think>', text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

In [20]:
resultant_dataframe['expalanation'] = resultant_dataframe['prediction'].apply(extract_explanation)

In [21]:
import re


def retrieve_answer_span(text):
    pattern = r"</think>\s*(.*)"
    match = re.search(pattern, text, re.DOTALL)
    result = match.group(1) if match else None
    return result


In [22]:
resultant_dataframe['pre_answer'] = resultant_dataframe['prediction'].apply(retrieve_answer_span)

In [23]:
resultant_dataframe['pre_answer']

0      The current CEO of Nvidia is Jensen Huang.\n\n...
1      Jean-Laurent Bonnafé is the current CEO of BNP...
2      Jack Dorsey is the current CEO of Block, Inc.\...
3      Mukesh D. Ambani is the current CEO of Relianc...
4      The current CEO of Frasers Group is Michael Mu...
                             ...                        
340                                           Ömer Tetik
341    Ye Guofu is the current CEO of Miniso.\n\nAnsw...
342    Dr. Tobias Meyer is the current CEO of Deutsch...
343    The current CEO of Intel is Lip-Bu Tan.  \nCon...
344    The current CEO of Commonwealth Bank is Matt C...
Name: pre_answer, Length: 345, dtype: object

In [24]:
def extract_answer(text):
    pattern=r"(?:Answer:\s*[**]*\s*|\*\*Answer:\*\*\s*|The answer is:\s*|The answer label is: |The current CEO of .*? is \*\*|The current .*? is [**]*)\s*(?:<[^>]+>\s*)*([\w\s,.'-]+?)(?=\s*(?:\*\*\s*\.\s*\n\n|Confidence|$))"
    #pattern=r"(?:Answer:\s*[**]*\s*|The answer is:\s*|The current .*? is [**]*|</think>\s*\n+\s*)\s*(?:<[^>]+>\s*)*([\w\s,.-]+?)(?=\s*(?:\*\*\s*\.\s*\n\n|Confidence|$))"
   
    matches = re.findall(pattern, text, re.IGNORECASE | re.MULTILINE)
    last_match = matches[-1] if matches else None
    return last_match


In [25]:
resultant_dataframe['answer'] = resultant_dataframe['pre_answer'].apply(extract_answer)

In [26]:
resultant_dataframe['answer']

0              Jensen Huang
1      Jean-Laurent Bonnafé
2               Jack Dorsey
3          Mukesh D. Ambani
4            Michael Murray
               ...         
340                    None
341                Ye Guofu
342        Dr. Tobias Meyer
343             Lip-Bu Tan.
344              Matt Comyn
Name: answer, Length: 345, dtype: object

In [27]:
import re

    
def second_retrieve(text):
    pattern=r"^([A-Za-zÀ-ÖØ-öø-ÿ]+(?: [A-Za-zÀ-ÖØ-öø-ÿ\.\-]+)*?)(?=\s*(?:\n|Confidence|is the current|$))"
    #pattern = r'''(?:\*\*([^*]+)\*\*|^([A-Za-zÀ-ÖØ-öø-ÿ]+(?:\s[A-Za-zÀ-ÖØ-öø-ÿ.\-]+)*))(?=\s*(?:\n|Confidence|is the current|$))'''
    match = re.search(pattern, text, re.IGNORECASE | re.MULTILINE)

    if match:
        return match.group(1)
    return None
    

In [28]:
interm = resultant_dataframe[resultant_dataframe['answer'].isnull()]
interm['answer']= interm['pre_answer'].apply(second_retrieve)
interm[interm['answer'].isnull()]

/tmp/ipykernel_883/2482192990.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interm['answer']= interm['pre_answer'].apply(second_retrieve)


,Unnamed: 0,companyLabel,ceoLabel,prediction,confidence,expalanation,pre_answer,answer
29,29,Q9265848,Krzysztof Brzeziński,"<think>\nOkay, so I need to figure out who the...",Low,"Okay, so I need to figure out who the current ...",The current CEO of Q9265848 cannot be determin...,None
50,50,Q3077586,Chen Rui,"<think>\nOkay, so I need to figure out who the...",low,"Okay, so I need to figure out who the current ...",The current CEO of Q3077586 cannot be determin...,None
79,79,Songa Offshore SE,Bjørnar Iversen,"<think>\nAlright, let's try to figure out who ...",medium,"Alright, let's try to figure out who the curre...",The most recent information indicates that aft...,None
133,133,Norwegian Air Shuttle,Geir Karlsen,"<think>\nOkay, I need to figure out who the cu...",high,"Okay, I need to figure out who the current CEO...",The current CEO of Norwegian Air Shuttle is **...,None
143,143,Fortescue Metals Group,Elizabeth Gaines,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...",The current CEO of Fortescue Metals Group is *...,None
147,147,HP Inc.,Dion Weisler,"<think>\nOkay, I need to figure out who the cu...",high,"Okay, I need to figure out who the current CEO...",The current CEO of HP Inc. is **Enrique Lores*...,None
149,149,Caterpillar Inc.,Jim Umpleby,"<think>\nOkay, so I need to figure out who the...",High,"Okay, so I need to figure out who the current ...",The current CEO of Caterpillar Inc. is D. Jame...,None
151,151,Orange Polska,Bruno Duthoit,"<think>\nOkay, so I need to figure out who the...",High,"Okay, so I need to figure out who the current ...",The current CEO of Orange Polska is **Liudmila...,None
157,157,Alphabet Inc.,Sundar Pichai,"<think>\nOkay, so I need to figure out who the...",high,"Okay, so I need to figure out who the current ...",The current CEO of Alphabet Inc. is **Sundar P...,None
159,159,ADA,Nicolas Rousselet,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...","Charles D. ""Chuck"" Henderson is the current CE...",None


In [29]:
resultant_dataframe[resultant_dataframe['answer'].isnull()]['pre_answer'].apply(second_retrieve)

21                 Elad Aharonson
24              Charles W. Scharf
29                           None
34                  Peter Zaffino
43              Damon T. Hininger
                  ...            
325               Helen Lofthouse
333            Ignacio Madridejos
335                 Jaime Ramirez
336    Juan López-Belmonte Encina
340                    Ömer Tetik
Name: pre_answer, Length: 73, dtype: object

In [34]:
resultant_dataframe.columns

Index(['Unnamed: 0', 'companyLabel', 'ceoLabel', 'prediction', 'confidence',
       'expalanation', 'pre_answer', 'answer'],
      dtype='object')

In [35]:
resultant_dataframe.drop(columns=['Unnamed: 0','prediction', 'confidence','expalanation']).to_csv("ceo_deepseek_with_web_processed.csv",index=False)

In [42]:
df=pd.read_csv("ceo_deepseek_with_web_processed.csv")

In [43]:
df[df['answer'].isnull()]

,companyLabel,ceoLabel,pre_answer,answer


In [265]:
resultant_dataframe[resultant_dataframe['ceoLabel']!=resultant_dataframe['answer']]

,Unnamed: 0,companyLabel,ceoLabel,prediction,confidence,explanation,answer
0,0,Nvidia,Jensen Huang,"<think>\nOkay, so I need to figure out who the...",High,"Okay, so I need to figure out who the current ...",Jensen Huang.
3,3,Reliance Industries Limited,Mukesh Ambani,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...",Mukesh D. Ambani
4,4,Frasers Group,Mike Ashley,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...",correct.
5,5,Lazard,Peter R. Orszag,"<think>\nAlright, so I need to figure out who ...",High,"Alright, so I need to figure out who the curre...",Peter Orszag
7,7,Lenovo,Yang Yuanqing,"<think>\nOkay, so I need to figure out who the...",High,"Okay, so I need to figure out who the current ...",Yuanqing Yang.
...,...,...,...,...,...,...,...
339,339,Tessellis,Davide Rota,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...","outdated since it refers to the founder, which..."
340,340,Banca Transilvania,Ömer Tetik,"<think>\nAlright, so I need to figure out who ...",NaN,"Alright, so I need to figure out who the curre...",None
342,342,Deutsche Post AG,Tobias Meyer,"<think>\nOkay, so I need to figure out who the...",high,"Okay, so I need to figure out who the current ...",Dr. Tobias Meyer.
343,343,Intel,Michelle Johnston Holthaus,"<think>\nOkay, I need to figure out who the cu...",High,"Okay, I need to figure out who the current CEO...",based on the provided search results. Let me g...


In [9]:
from entity_align import extract_answer

/root/autovu/venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [12]:
from fuzzywuzzy import fuzz 

def rowwise_eval(row1, row2, threshold=75):
    score = fuzz.ratio(str(row1).lower(), str(row2).lower())
    return True if score >= threshold else False



In [1]:
import pandas as pd 

deepseek = pd.read_csv('bank_governors_results_deepseek.csv')

In [2]:
df_without_web_llama=pd.read_csv("bank_governors_results_without_web_llama.csv")

In [3]:
df_no_web_deepseek= pd.read_csv("bank_governors_results_without_web_deepseek.csv")

In [4]:
df_llama = pd.read_csv("bank_governors_results_llama.csv")

In [5]:
deepseek['eval'].mean()

0.9545454545454546

In [6]:
df_without_web_llama['eval'].mean()

0.5217391304347826

In [7]:
df_no_web_deepseek['eval'].mean()

0.5217391304347826

In [8]:
df_llama['eval'].mean()

0.9565217391304348

In [ ]:
# df_llama['eval'] = [
#     rowwise_eval(a1, a2) if e1 is True else None
#     for a1, a2, e1 in zip(df['answer'], df_llama['answer'], df['eval'])
# ]


In [ ]:
# df_without_web_llama['eval'] = [
#     rowwise_eval(a1, a2) if e1 is True else None
#     for a1, a2, e1 in zip(df['answer'], df_without_web_llama['answer'], df['eval'])
# ]


In [ ]:
# df_no_web_deepseek['eval'] = [
#     rowwise_eval(a1, a2) if e1 is True else None
#     for a1, a2, e1 in zip(df['answer'], df_no_web_deepseek['answer'], df['eval'])
# ]


In [10]:
df = pd.read_csv('data/company_ceo_31-03.csv')
df

,companyLabel,ceoLabel
0,Nvidia,Jensen Huang
1,BNP Paribas,Jean-Laurent Bonnafé
2,"Block, Inc.",Jack Dorsey
3,Reliance Industries Limited,Mukesh Ambani
4,Frasers Group,Mike Ashley
...,...,...
340,Banca Transilvania,Ömer Tetik
341,Miniso,Ye Guofu
342,Deutsche Post AG,Tobias Meyer
343,Intel,Michelle Johnston Holthaus


In [18]:
import pandas as pd
df_llama=pd.read_csv("results/comp_ceo_results_llama.csv")
df_deepseek=pd.read_csv("results/ceo_deepseek_with_web_processed.csv")
df_no_web_deepseek=pd.read_csv("results/comp_ceo_results_without_web_deepseek.csv")
df_no_web_llama=pd.read_csv("results/comp_ceo_results_without_web_llama.csv")

In [6]:
df_no_web_deepseek['eval'].mean()

0.4378698224852071

In [7]:
df_deepseek['eval'].mean()

0.9260355029585798

In [8]:
df_llama['eval'].mean()

0.8668639053254438

In [23]:
from entity_align import extract_answer

In [ ]:
#df_no_web_llama['eval'] = df_no_web_llama['eval'].map({'True': True, 'False': False})

In [9]:
df_no_web_llama['eval'].mean()

0.456973293768546

In [ ]:
# # Step 1: Merge df and df_llama based on 'answer'
# df_ref = df_deepseek[['companyLabel']]  # or whichever columns you want to align
# df_llama_aligned = df_ref.merge(df_llama[['answer', 'eval', 'companyLabel']], on='companyLabel', how='left')

In [ ]:
# df_llama_aligned[df_llama_aligned['answer'].isnull()]

,companyLabel,answer,eval
36,Q24941022,NaN,NaN
57,Q34120381,NaN,NaN
97,Tampella,NaN,NaN
189,Mylan,NaN,NaN


In [68]:
df_with_web = pd.read_csv("national_football_teams_results_llama.csv")

In [70]:
df_with_web['eval'].mean()

0.7959183673469388

In [73]:
df_no_web = pd.read_csv("national_football_teams_no_web_results_llama.csv")

In [74]:
df_no_web['eval'].mean()

0.08163265306122448

In [1]:
from autovu import answer_with_gpt

/root/autovu/autovu.py:35: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(


In [2]:
answer_with_gpt("Who is the current CEO of Intel?")

'Answer: Lip-Bu Tan\nConfidence: high\nReference: https://en.wikipedia.org/wiki/Pat_Gelsinger '

In [7]:
import pandas as pd

df = pd.read_csv("company_ceo_31-03_results_gpt.csv")

In [ ]:
# import pandas as pd
# import re

# def extract_fields(text):
#     pattern = (
#         r"Answer:\s*(?P<answer>.*?)\s*"
#         r"Confidence:\s*(?P<confidence>.*?)\s*"
#         r"Reference:\s*(?P<reference>.*?)\s*$"
#     )
#     match = re.search(pattern, text, re.DOTALL)
#     if match:
#         return match.group("answer"), match.group("confidence"), match.group("reference")
#     else:
#         return None, None, None

# df[['answer', 'confidence', 'reference']] = df['prediction'].astype(str).apply(
#     lambda x: pd.Series(extract_fields(x))
# )

In [ ]:
#df.to_csv("company_ceo_31-03_results_gpt.csv", index=False)

In [ ]:
# df['eval'] = [
#     rowwise_eval(a1, a2) if e1 is True else None
#     for a1, a2, e1 in zip(df_llama['answer'], df['answer'], df_llama['eval'])
# ]


In [8]:
df[df['eval']==False]

,companyLabel,ceoLabel,confidence,answer,eval,reference
13,21st Century Fox,James Murdoch,high,James Murdoch,False,([cnas.org](https://www.cnas.org/press/press-r...
14,Sears,Eddie Lampert,high,Eddie Lampert,False,([moneyinc.com](https://moneyinc.com/sears-ceo...
42,SalMar,Gustav Witzøe,High,Frode Arntsen,False,([seekingalpha.com](https://seekingalpha.com/a...
43,Odakyu Electric Railway,Kōji Hoshino,high,Koji Hoshino,False,([financecharts.com](https://www.financecharts...
86,Rotem Shani,Q17522859,high,Yehuda Yedidia,False,([marketscreener.com](https://www.marketscreen...
96,Imperial Hotel,Hideya Sadayasu,high,Hideya Sadayasu,False,([imperialhotel.co.jp](https://www.imperialhot...
107,Maggie Beer Holdings,Laura McBain,medium,Kinda Grange,False,([foodanddrinkbusiness.com.au](https://www.foo...
117,Homeserve,Richard Harpin,high,Richard Harpin,False,([en.wikipedia.org](https://en.wikipedia.org/w...
136,Nokia,Björn Westerlund,high,Pekka Lundmark,False,([fortune.com](https://fortune.com/europe/2023...
138,KGHM Polska Miedź,Radosław Domagalski-Łabędzki,High,Tomasz Zdzikot,False,([minexforum.com](https://minexforum.com/2023/...


In [9]:
df['eval'].mean()

0.9112426035502958

In [ ]:
#df.to_csv("company_ceo_31-03_results_gpt.csv", index=False)

In [16]:
import pandas as pd
df_llama=pd.read_csv("results/comp_ceo_results_llama.csv")
df_deepseek=pd.read_csv("results/ceo_deepseek_with_web_processed.csv")

In [ ]:
from entity_align import fuzzy_match


#df_deepseek['outdated'] = df_deepseek.apply(lambda x: not(fuzzy_match(x['ceoLabel'], x['answer'])) if x['eval'] else None, axis=1)

In [17]:
df_deepseek[df_deepseek['outdated'].isnull()]

,companyLabel,ceoLabel,pre_answer,answer,eval,outdated


In [19]:
df_deepseek['outdated'].mean()

0.46449704142011833

In [ ]:
#df_deepseek.to_csv("results/ceo_deepseek_with_web_processed.csv", index=False)

In [21]:
df_llama = pd.read_csv("results/bank_governors_results_llama.csv")

In [22]:
len(df_llama[df_llama['outdated']==True])

12

In [23]:
df_llama['outdated'].mean()

0.5217391304347826

In [61]:
df = pd.read_csv("bank_governors_01-04_results_gpt.csv")

In [37]:
import pandas as pd
import re

def extract_fields(text):
    pattern = (
        r"Answer:\s*(?P<answer>.*?)\s*"
        r"Confidence:\s*(?P<confidence>.*?)\s*"
        r"Reference:\s*(?P<reference>.*?)\s*$"
    )
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group("answer"), match.group("confidence"), match.group("reference")
    else:
        return None, None, None

df[['answer', 'confidence', 'reference']] = df['prediction'].astype(str).apply(
    lambda x: pd.Series(extract_fields(x))
)

In [ ]:
# df['eval'] = [
#     rowwise_eval(a1, a2) if e1 is True else None
#     for a1, a2, e1 in zip(df_llama['answer'], df['answer'], df_llama['eval'])
# ]


In [ ]:
#df.to_csv("bank_governors_01-04_results_gpt.csv",index=False)

In [62]:
df.columns

Index(['bankLabel', 'governorLabel', 'answer', 'eval', 'confidence',
       'positionLabel', 'reference'],
      dtype='object')

In [ ]:
#df = df.drop(columns=['prediction', 'startDate', 'bank', 'position', 'governor'])

In [ ]:
#df = df.iloc[:, [0,2,3,6,4,1,5]]

In [57]:
df

,bankLabel,governorLabel,answer,eval,confidence,positionLabel,reference
0,National Bank of Romania,Mugur Isărescu,Mugur Isărescu,True,high,Governor of the National Bank of Romania,([bloomberg.com](https://www.bloomberg.com/new...
1,Bank of Finland,Olli Rehn,Olli Rehn,True,high,Governor of the Bank of Finland,([suomenpankki.fi](https://www.suomenpankki.fi...
2,European Central Bank,Christine Lagarde,Christine Lagarde is the President of the Euro...,False,High,President of the European Central Bank,([en.wikipedia.org](https://en.wikipedia.org/w...
3,Bank of England,beginning,Andy Beshear,False,high,Governor of the Bank of England,([governor.ky.gov](https://governor.ky.gov/abo...
4,Bank of Greece,Yannis Stournaras,Yannis Stournaras,True,high,Governor of the Bank of Greece,([bis.org](https://www.bis.org/review/r241211h...
5,Bank of France,Christian Noyer,François Villeroy de Galhau,True,High,Governor of the Banque de France,https://en.wikipedia.org/wiki/Christian_Noyer
6,Central Bank of the Republic of Turkey,Süreyya Serdengeçti,Fatih Karahan,True,High,Governor of the Central Bank of Turkey,([tcmb.gov.tr](https://tcmb.gov.tr/wps/wcm/con...
7,People's Bank of China,Pan Gongsheng,Pan Gongsheng,True,high,Governor of the People's Bank of China,([en.wikipedia.org](https://en.wikipedia.org/w...
8,State Bank of Vietnam,Cao Sĩ Kiêm,Cao Sĩ Kiêm,False,high,Governor of the State Bank of Vietnam,([infonet.vietnamnet.vn](https://infonet.vietn...
9,Bank of Japan,Kazuo Ueda,NaN,False,NaN,governor of the Bank of Japan,NaN


In [58]:
df.to_csv("bank_governors_01-04_results_gpt.csv", index=False)

In [60]:
df[df['eval'].isnull()]

,bankLabel,governorLabel,answer,eval,confidence,positionLabel,reference


In [63]:
df['eval'].mean()

0.5217391304347826

In [2]:
import pandas as pd

df = pd.read_csv("data/company_ceo_31-03.csv")

In [3]:
len(df)

339

In [4]:
df_1 = pd.read_csv("data/bank_governors_01-04.csv")

In [6]:
len(df_1)

23

In [7]:
df_2 = pd.read_csv("data/international_organizations_leadership_31-03.csv")

In [8]:
len(df_2)

325

In [10]:
with open('data/relations.txt', 'r') as f:
    lines = f.readlines()
    relations = [line.strip() for line in lines]

In [12]:
len(relations)

40

In [4]:
from utils import entity2id_codex


input_file = "data/humans_wikidata/humans_wikidata/type2relation2type_ttv.txt"

unique_relations = set()

with open(input_file, "r", encoding="utf-8") as f:
    for line in f:
        parts = line.strip().split("\t")
        if len(parts) >= 2:
            relation = parts[1].strip()
            if "/relation/" in relation:
                rel_id = relation.split("/")[-1]
                unique_relations.add(rel_id)

sorted_relations = sorted(unique_relations)

entities_dict, relations_dict = entity2id_codex()
human_wikidata_relations = []

for rel in sorted_relations:
    relation = relations_dict.get(rel, None)
    if relation:
        human_wikidata_relations.append(relation)

In [6]:
len(human_wikidata_relations)

41

In [7]:
human_wikidata_relations

['field of work',
 'member of political party',
 'medical condition',
 'occupation',
 'employer',
 'founded by',
 'place of burial',
 'publisher',
 'instrument',
 'located in the administrative terroritorial entity',
 'movement',
 'genre',
 'named after',
 'religion',
 'languages spoken, written, or signed',
 'country',
 'creator',
 'ethnic group',
 'place of birth',
 'place of death',
 'time period',
 'spouse',
 'record label',
 'country of citizenship',
 'practiced by',
 'sibling',
 'part of',
 'child',
 'unmarried partner',
 'member of',
 'chairperson',
 'country of origin',
 'cause of death',
 'member of sports team',
 'residence',
 'director',
 'sport',
 'educated at',
 'influenced by',
 'location of formation',
 'notable works']

In [2]:
import pandas as pd

df=pd.read_csv("results/ceo_deepseek_with_web_processed.csv")

In [3]:
df

,companyLabel,ceoLabel,pre_answer,answer,eval,outdated
0,Nvidia,Jensen Huang,The current CEO of Nvidia is Jensen Huang.\n\n...,Jensen Huang,True,False
1,BNP Paribas,Jean-Laurent Bonnafé,Jean-Laurent Bonnafé is the current CEO of BNP...,Jean-Laurent Bonnafé,True,False
2,"Block, Inc.",Jack Dorsey,"Jack Dorsey is the current CEO of Block, Inc.\...",Jack Dorsey,True,False
3,Reliance Industries Limited,Mukesh Ambani,Mukesh D. Ambani is the current CEO of Relianc...,Mukesh D. Ambani,True,False
4,Frasers Group,Mike Ashley,The current CEO of Frasers Group is Michael Mu...,Michael Murray,True,True
...,...,...,...,...,...,...
333,Banca Transilvania,Ömer Tetik,Ömer Tetik,Ömer Tetik,True,False
334,Miniso,Ye Guofu,Ye Guofu is the current CEO of Miniso.\n\nAnsw...,Ye Guofu,True,False
335,Deutsche Post AG,Tobias Meyer,Dr. Tobias Meyer is the current CEO of Deutsch...,Dr. Tobias Meyer,True,False
336,Intel,Michelle Johnston Holthaus,The current CEO of Intel is Lip-Bu Tan. \nCon...,Lip-Bu Tan.,True,True


In [4]:
df['confidence']=df['pre_answer'].str.extract(r'(?i)\b(high|medium|low)\b')

In [5]:
df

,companyLabel,ceoLabel,pre_answer,answer,eval,outdated,confidence
0,Nvidia,Jensen Huang,The current CEO of Nvidia is Jensen Huang.\n\n...,Jensen Huang,True,False,High
1,BNP Paribas,Jean-Laurent Bonnafé,Jean-Laurent Bonnafé is the current CEO of BNP...,Jean-Laurent Bonnafé,True,False,High
2,"Block, Inc.",Jack Dorsey,"Jack Dorsey is the current CEO of Block, Inc.\...",Jack Dorsey,True,False,High
3,Reliance Industries Limited,Mukesh Ambani,Mukesh D. Ambani is the current CEO of Relianc...,Mukesh D. Ambani,True,False,High
4,Frasers Group,Mike Ashley,The current CEO of Frasers Group is Michael Mu...,Michael Murray,True,True,High
...,...,...,...,...,...,...,...
333,Banca Transilvania,Ömer Tetik,Ömer Tetik,Ömer Tetik,True,False,NaN
334,Miniso,Ye Guofu,Ye Guofu is the current CEO of Miniso.\n\nAnsw...,Ye Guofu,True,False,High
335,Deutsche Post AG,Tobias Meyer,Dr. Tobias Meyer is the current CEO of Deutsch...,Dr. Tobias Meyer,True,False,high
336,Intel,Michelle Johnston Holthaus,The current CEO of Intel is Lip-Bu Tan. \nCon...,Lip-Bu Tan.,True,True,High


In [13]:
df[(df['eval'] == True) & (df['confidence'] == 'medium')]


,companyLabel,ceoLabel,pre_answer,answer,eval,outdated,confidence
130,Tauron Polska Energia,Filip Grzegorczyk,The current CEO of Tauron Polska Energia is Pa...,Pawel Straczynski,True,True,medium
131,Viaplay Group,Anders Jensen,Jørgen Madsen Lindemann is the current CEO of ...,Jørgen Madsen Lindemann,True,True,medium
140,GMM Grammy,Boosaba Daorueng,The current CEO of GMM Grammy is Boosaba Daoru...,Boosaba Daorueng,True,False,medium
161,FNC Entertainment,Han Sung-ho,The current CEOs of FNC Entertainment are Ahn ...,Ahn Seok-joon and Han Seung-hoon,True,True,medium
220,Repsol Oil & Gas Canada,James Buckee,Josu Jon Imaz San Miguel is the current CEO of...,Josu Jon Imaz San Miguel,True,True,medium
311,WiseTech Global Limited,Richard White,The current acting CEO of WiseTech Global Limi...,Andrew Cartledge,True,True,medium


In [18]:

import pandas as pd 

df=pd.read_csv("results/company_ceo_31-03_results_gpt.csv")

In [ ]:
#df['confidence']=df['prediction'].str.extract(r'(?i)\b(high|medium|low)\b')

In [20]:
len(df[df['eval']==False])

30

In [25]:
len(df[(df['eval'] == False) & (df['confidence'] == 'high')])


19

In [ ]:
# 12 high out of 45
#7+19

In [26]:
from autovu import answer_with_gpt
answer_with_gpt(f"Who is the current governor of Intel Corporation?")

'Answer: Lip-Bu Tan\n\nConfidence: high\n\nReference: ([newsroom.intel.com](https://newsroom.intel.com/corporate/tan-joins-intel-ceo?utm_source=openai)) '

In [1]:
import pandas as pd

df = pd.read_csv("international_organizations_leadership_31-03_results_llama.csv")

In [2]:
df

,organizationLabel,positionLabel,officeholderLabel,prediction
0,African Development Bank,Akinwumi Adesina,Akinwumi Adesina,"Answer: Akinwumi Adesina, Confidence: High."
1,African Paralympic Committee,President of the African Paralympic Committee,Leonel da Rocha Pinto,"Answer: Samson Deen, Confidence: High."
2,Afro-Asian People's Solidarity Organization,Anwar Sadat,Anwar Sadat,"Answer: Dr. Helmi Hadidi, Confidence: medium"
3,Aga Khan Development Network,Ismaili imam,Ismaili imam,"Answer: His Highness Prince Rahim Aga Khan V, ..."
4,All European Academies,Antonio Loprieno,Antonio Loprieno,"Answer: Paweł Rowiński, Confidence: high."
...,...,...,...,...
320,World Women’s Snooker,Mandy Fisher,Mandy Fisher,"Answer: Mandy Fisher, Confidence: medium"
321,World Zionist Organization,Yaakov Hagoel,Yaakov Hagoel,"Answer: Tova Dorfman, Confidence: high"
322,World Baseball Softball Confederation,Riccardo Fraccari,Riccardo Fraccari,"Answer: Riccardo Fraccari, Confidence: high"
323,Young Christian Workers,Laetitia M. Navarro,Laetitia M. Navarro,"Answer: Basma Louis, Confidence: medium"


In [3]:
df['Confidence'] = df['prediction'].str.extract(r'(?i)\b(high|medium|low)\b')

In [5]:
from entity_align import extract_answer

df['answer'] = df['prediction'].apply(lambda x: extract_answer(x))

In [6]:
df

,organizationLabel,positionLabel,officeholderLabel,prediction,Confidence,answer
0,African Development Bank,Akinwumi Adesina,Akinwumi Adesina,"Answer: Akinwumi Adesina, Confidence: High.",High,Akinwumi Adesina
1,African Paralympic Committee,President of the African Paralympic Committee,Leonel da Rocha Pinto,"Answer: Samson Deen, Confidence: High.",High,Samson Deen
2,Afro-Asian People's Solidarity Organization,Anwar Sadat,Anwar Sadat,"Answer: Dr. Helmi Hadidi, Confidence: medium",medium,Dr. Helmi Hadidi
3,Aga Khan Development Network,Ismaili imam,Ismaili imam,"Answer: His Highness Prince Rahim Aga Khan V, ...",high,His Highness Prince Rahim Aga Khan V
4,All European Academies,Antonio Loprieno,Antonio Loprieno,"Answer: Paweł Rowiński, Confidence: high.",high,Paweł Rowiński
...,...,...,...,...,...,...
320,World Women’s Snooker,Mandy Fisher,Mandy Fisher,"Answer: Mandy Fisher, Confidence: medium",medium,Mandy Fisher
321,World Zionist Organization,Yaakov Hagoel,Yaakov Hagoel,"Answer: Tova Dorfman, Confidence: high",high,Tova Dorfman
322,World Baseball Softball Confederation,Riccardo Fraccari,Riccardo Fraccari,"Answer: Riccardo Fraccari, Confidence: high",high,Riccardo Fraccari
323,Young Christian Workers,Laetitia M. Navarro,Laetitia M. Navarro,"Answer: Basma Louis, Confidence: medium",medium,Basma Louis


In [7]:
df.to_csv("international_organizations_leadership_31-03_results_llama.csv", index=False)

In [9]:
new_order = ['organizationLabel', 'officeholderLabel','answer', 'Confidence']
df = df[new_order]


In [10]:
df

,organizationLabel,officeholderLabel,answer,Confidence
0,African Development Bank,Akinwumi Adesina,Akinwumi Adesina,High
1,African Paralympic Committee,Leonel da Rocha Pinto,Samson Deen,High
2,Afro-Asian People's Solidarity Organization,Anwar Sadat,Dr. Helmi Hadidi,medium
3,Aga Khan Development Network,Ismaili imam,His Highness Prince Rahim Aga Khan V,high
4,All European Academies,Antonio Loprieno,Paweł Rowiński,high
...,...,...,...,...
320,World Women’s Snooker,Mandy Fisher,Mandy Fisher,medium
321,World Zionist Organization,Yaakov Hagoel,Tova Dorfman,high
322,World Baseball Softball Confederation,Riccardo Fraccari,Riccardo Fraccari,high
323,Young Christian Workers,Laetitia M. Navarro,Basma Louis,medium


In [3]:

import pandas as pd

df = pd.read_csv("international_organizations_leadership_31-03_results_llama.csv")

In [5]:
df['eval'].mean()

0.7009345794392523

In [6]:
df.columns

Index(['organizationLabel', 'officeholderLabel', 'answer', 'Confidence',
       'eval'],
      dtype='object')

In [7]:
from entity_align import fuzzy_match


df['outdated'] = df.apply(lambda x: not(fuzzy_match(x['officeholderLabel'], x['answer'])) if x['eval'] else None, axis=1)

/root/autovu/venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [14]:
df_llama= pd.read_csv("international_organizations_leadership_31-03_results_llama_no_web.csv")
df_r1 = pd.read_csv("international_organizations_leadership_31-03_results_r1.csv")

In [18]:
from entity_align import extract_answer

df_r1['answer']=df_r1['prediction'].astype(str).apply(lambda x: extract_answer(x))

In [20]:
df_r1['confidence']=df_r1['prediction'].str.extract(r'(?i)\b(high|medium|low)\b')

In [23]:
df_r1['eval'] = [
    rowwise_eval(a1, a2) if e1 is True else None
    for a1, a2, e1 in zip(df['answer'], df_r1['answer'], df['eval'])
]


In [26]:
df_r1 = df_r1[["organizationLabel", "officeholderLabel","prediction", "confidence", "answer", "eval"]]

In [28]:
df_r1.drop(columns=['prediction']).to_csv("international_org_ann.csv",index=False)

In [27]:
df_r1.to_csv("international_organizations_leadership_31-03_results_r1.csv", index=False)

In [12]:
df[df['outdated']==True]

,organizationLabel,officeholderLabel,answer,Confidence,eval,outdated
1,African Paralympic Committee,Leonel da Rocha Pinto,Samson Deen,High,True,True
3,Aga Khan Development Network,Ismaili imam,His Highness Prince Rahim Aga Khan V,high,True,True
5,Alliance of Passenger Rail New Entrants in Europe,Erich Forster,Elmer van Buuren,high,True,True
7,American Association for the Advancement of Sc...,Eloise E. Clark,Marina Picciotto,high,True,True
10,Arab League,Secretary General of the Arab League,Ahmed Aboul Gheit,high,True,True
...,...,...,...,...,...,...
305,World Federation of United Nations Associations,Park Soo-gil,Dr. Shawn Chen,High,True,True
309,World League for Freedom and Democracy,Zhao Ziqi,Yao Eng-chi,medium,True,True
310,World Mountain Running Association,Jonathan Wyatt,Tomo Šarf,High,True,True
318,World Uyghur Congress,Dolkun Isa,Turghunjan Alawudun,high,True,True


In [ ]:
from entity_align import fuzzy_match


#df_deepseek['outdated'] = df_deepseek.apply(lambda x: not(fuzzy_match(x['ceoLabel'], x['answer'])) if x['eval'] else None, axis=1)

In [ ]:
#df.to_csv("international_organizations_leadership_31-03_results_llama.csv", index=False)

In [13]:
import pandas as pd

df = pd.read_csv("international_org_ann_r1.csv")

In [14]:
df['eval'].mean()

0.7253086419753086

In [32]:
len(df[df['eval'] == False])


89

In [24]:
df_deepseek_no_web = pd.read_csv("international_organizations_leadership_31-03_results_r1_no_web.csv")

In [26]:
df_deepseek_no_web['confidence'] = df_deepseek_no_web['prediction'].str.extract(r'(?i)\b(high|medium|low)\b')

In [28]:

from entity_align import extract_answer
df_deepseek_no_web['answer'] = df_deepseek_no_web['prediction'].apply(lambda x: extract_answer(x))

In [34]:

df_deepseek_no_web['eval'] = [
    rowwise_eval(a1, a2) if e1 is True else None
    for a1, a2, e1 in zip(df['answer'], df_deepseek_no_web['answer'], df['eval'])
]


In [35]:
df_deepseek_no_web['eval']

0       True
1       True
2      False
3      False
4       None
       ...  
320     None
321    False
322     True
323     None
324    False
Name: eval, Length: 325, dtype: object

In [1]:
import pandas as pd

df_deepseek_no_web = pd.read_csv("international_org_ann_r1_no_web.csv")

In [3]:
df_deepseek_no_web['eval'].mean()

0.27384615384615385

In [4]:
df_llama_no_web = pd.read_csv("international_organizations_leadership_31-03_results_llama_no_web.csv")

In [5]:
df_llama_no_web

,organizationLabel,positionLabel,officeholderLabel,prediction
0,African Development Bank,Akinwumi Adesina,Akinwumi Adesina,"Answer: Akinwumi Adesina, Confidence: High."
1,African Paralympic Committee,President of the African Paralympic Committee,Leonel da Rocha Pinto,I don't have access to a search engine to prov...
2,Afro-Asian People's Solidarity Organization,Anwar Sadat,Anwar Sadat,I couldn't find any information on the current...
3,Aga Khan Development Network,Ismaili imam,Ismaili imam,"Answer: His Highness the Aga Khan IV, Confiden..."
4,All European Academies,Antonio Loprieno,Antonio Loprieno,"Answer: Antonio Loprieno, Confidence: medium"
...,...,...,...,...
320,World Women’s Snooker,Mandy Fisher,Mandy Fisher,"Mandy Fisher, but now it's Mink Nutcharut"
321,World Zionist Organization,Yaakov Hagoel,Yaakov Hagoel,"Answer: Yaakov Hagoel, Confidence: high."
322,World Baseball Softball Confederation,Riccardo Fraccari,Riccardo Fraccari,"Riccardo Fraccari, Confidence: high."
323,Young Christian Workers,Laetitia M. Navarro,Laetitia M. Navarro,"I apologize, but I cannot verify who the curre..."


In [6]:
from entity_align import extract_answer 

df_llama_no_web['confidence']=df_llama_no_web['prediction'].str.extract(r'(?i)\b(high|medium|low)\b')
df_llama_no_web['answer'] = df_llama_no_web['prediction'].apply(lambda x:extract_answer(x))

In [9]:
df_llama_no_web

,organizationLabel,positionLabel,officeholderLabel,prediction,confidence,answer
0,African Development Bank,Akinwumi Adesina,Akinwumi Adesina,"Answer: Akinwumi Adesina, Confidence: High.",High,Akinwumi Adesina
1,African Paralympic Committee,President of the African Paralympic Committee,Leonel da Rocha Pinto,I don't have access to a search engine to prov...,NaN,None
2,Afro-Asian People's Solidarity Organization,Anwar Sadat,Anwar Sadat,I couldn't find any information on the current...,NaN,None
3,Aga Khan Development Network,Ismaili imam,Ismaili imam,"Answer: His Highness the Aga Khan IV, Confiden...",high,His Highness the Aga Khan IV
4,All European Academies,Antonio Loprieno,Antonio Loprieno,"Answer: Antonio Loprieno, Confidence: medium",medium,Antonio Loprieno
...,...,...,...,...,...,...
320,World Women’s Snooker,Mandy Fisher,Mandy Fisher,"Mandy Fisher, but now it's Mink Nutcharut",NaN,None
321,World Zionist Organization,Yaakov Hagoel,Yaakov Hagoel,"Answer: Yaakov Hagoel, Confidence: high.",high,Yaakov Hagoel
322,World Baseball Softball Confederation,Riccardo Fraccari,Riccardo Fraccari,"Riccardo Fraccari, Confidence: high.",high,None
323,Young Christian Workers,Laetitia M. Navarro,Laetitia M. Navarro,"I apologize, but I cannot verify who the curre...",NaN,None


In [11]:
df_llama_no_web.drop(columns=['positionLabel','prediction']).to_csv("international_organizations_leadership_31-03_results_llama_no_web.csv", index=False)

In [16]:

df_llama_no_web['eval'] = [
    rowwise_eval(a1, a2) if e1 is True else None
    for a1, a2, e1 in zip(df['answer'], df_llama_no_web['answer'], df['eval'])
]


In [18]:
df_llama_no_web.to_csv("international_organizations_leadership_31-03_results_llama_no_web.csv", index=False)

In [19]:
df_llama_no_web.drop(columns=['positionLabel', 'prediction']).to_csv("international_organizations_leadership_31-03_results_llama_no_web.csv", index=False)

In [78]:
df_llama_no_web.query("organizationLabel=='Young Pirates of Europe'")


,organizationLabel,positionLabel,officeholderLabel,prediction,confidence,answer,eval
324,Young Pirates of Europe,Arnaldur Sigurðarson,Arnaldur Sigurðarson,I don't have access to a search engine to prov...,NaN,None,False


In [81]:
df_llama_no_web.to_csv("backup.csv",index=False)

In [82]:
df = pd.read_csv("international_organizations_leadership_31-03_results_llama_no_web.csv")

In [84]:
df['eval'].mean()

0.18885448916408668

In [2]:
import pandas as pd

df = pd.read_csv("international_organizations_leadership_31-03_results_gpt.csv")

In [4]:
import pandas as pd
import re

def extract_fields(text):
    pattern = (
        r"Answer:\s*(?P<answer>.*?)\s*"
        r"Confidence:\s*(?P<confidence>.*?)\s*"
        r"Reference:\s*(?P<reference>.*?)\s*$"
    )
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group("answer"), match.group("confidence"), match.group("reference")
    else:
        return None, None, None

df[['answer', 'confidence', 'reference']] = df['prediction'].astype(str).apply(
    lambda x: pd.Series(extract_fields(x))
)

In [5]:
df

,organizationLabel,positionLabel,officeholderLabel,prediction,answer,confidence,reference
0,African Development Bank,Akinwumi Adesina,Akinwumi Adesina,Answer: Akinwumi Adesina\nConfidence: high\nRe...,Akinwumi Adesina,high,([afdb.org](https://www.afdb.org/en/about-us/o...
1,African Paralympic Committee,President of the African Paralympic Committee,Leonel da Rocha Pinto,Answer: Samson Deen\nConfidence: High\nReferen...,Samson Deen,High,([paralympic.org](https://www.paralympic.org/n...
2,Afro-Asian People's Solidarity Organization,Anwar Sadat,Anwar Sadat,Answer: Dr. Helmy El-Hadidi\n\nConfidence: Hig...,Dr. Helmy El-Hadidi,High,([aapsorg.org](https://www.aapsorg.org/en/?utm...
3,Aga Khan Development Network,Ismaili imam,Ismaili imam,Answer: His Highness Prince Rahim Aga Khan V\n...,His Highness Prince Rahim Aga Khan V,high,([the.akdn](https://the.akdn/en/who-we-are?utm...
4,All European Academies,Antonio Loprieno,Antonio Loprieno,Answer: Paweł Rowiński\nConfidence: high\nRefe...,Paweł Rowiński,high,([allea.org](https://allea.org/allea-turns-30-...
...,...,...,...,...,...,...,...
320,World Women’s Snooker,Mandy Fisher,Mandy Fisher,Answer: Mandy Fisher\nConfidence: high\nRefere...,Mandy Fisher,high,([womenssnooker.com](https://www.womenssnooker...
321,World Zionist Organization,Yaakov Hagoel,Yaakov Hagoel,Answer: Tova Dorfman\nConfidence: High\nRefere...,Tova Dorfman,High,([wzo.org.il](https://www.wzo.org.il/page/offi...
322,World Baseball Softball Confederation,Riccardo Fraccari,Riccardo Fraccari,Answer: Riccardo Fraccari\nConfidence: high\nR...,Riccardo Fraccari,high,([en.wikipedia.org](https://en.wikipedia.org/w...
323,Young Christian Workers,Laetitia M. Navarro,Laetitia M. Navarro,Answer: Sarah Prenger\n\nConfidence: High\n\nR...,Sarah Prenger,High,([joci.org](https://joci.org/en/press-releases...


In [7]:
ref_io = pd.read_csv("exp_results/international_org_ann_r1.csv")
ref_bg = pd.read_csv("exp_results/bank_governors_results_deepseek.csv")
ref_ceo =pd.read_csv("exp_results/ceo_deepseek_with_web_processed.csv")


In [34]:
import pandas as pd

bg_gpt_no_web=pd.read_csv("exp_results/bank_governors_01-04_results_gpt_no_web.csv")
ceo_gpt_no_web=pd.read_csv("exp_results/company_ceo_31-03_results_gpt_no_web.csv")
io_gpt_no_web=pd.read_csv("exp_results/international_organizations_leadership_31-03_results_gpt_no_web.csv")

In [35]:
from entity_align import extract_answer

io_gpt_no_web['confidence'] = io_gpt_no_web['prediction'].str.extract(r'(?i)\b(high|medium|low)\b')
io_gpt_no_web['answer'] = io_gpt_no_web['prediction'].apply(lambda x: extract_answer(x))

In [ ]:
#inter =ceo_gpt_no_web.merge(ref_ceo[['companyLabel', 'answer']], on='companyLabel', how='left')

In [36]:
io_gpt_no_web['eval'] = [
    rowwise_eval(a1, a2) if e1 is True else None
    for a1, a2, e1 in zip(ref_io['answer'], io_gpt_no_web['answer'], ref_io['eval'])
]


In [38]:
io_gpt_no_web[['organizationLabel','officeholderLabel', 'confidence', 'answer', 'eval']].to_csv("io_gpt_no_web.csv", index=False)

In [15]:
df = pd.read_csv("int_org_31-03_results_gpt.csv")

In [17]:
df['eval'].mean()

0.7337461300309598

In [18]:
len(df[df['eval']==False])

86

In [21]:
len(df[(df['eval'] == False) & (df['confidence'].str.lower() == 'low')])

0

In [2]:
import pandas as pd

bg_r1=pd.read_csv("exp_results/bank_governors_results_deepseek.csv")
bg_llama=pd.read_csv("exp_results/bank_governors_results_llama.csv")
bg_gpt=pd.read_csv("exp_results/bank_governors_01-04_gpt_web.csv")
ceo_r1=pd.read_csv("exp_results/ceo_deepseek_with_web_processed.csv")
ceo_llama=pd.read_csv("exp_results/comp_ceo_results_llama.csv")
ceo_gpt=pd.read_csv("exp_results/company_ceo_31-03_results_gpt.csv")
io_r1=pd.read_csv("exp_results/international_org_ann_r1.csv")
io_llama=pd.read_csv("exp_results/international_organizations_leadership_31-03_results_llama.csv")
io_gpt=pd.read_csv("exp_results/int_org_31-03_results_gpt.csv")

In [3]:
bg_r1['eval'].mean()

0.9545454545454546

In [4]:
ceo_r1['eval'].mean()

0.9349112426035503

In [5]:
ceo_llama['eval'].mean()

0.8668639053254438

In [6]:
ceo_gpt['eval'].mean()

0.9112426035502958

In [7]:
io_r1['eval'].mean()

0.7253086419753086

In [8]:
io_llama['eval'].mean()

0.7009345794392523

In [34]:
io_gpt['eval'].mean()

0.7337461300309598

In [13]:
io_gpt = io_gpt.merge(io_r1[['organizationLabel', 'outdated']], on = 'organizationLabel', how = 'left')

In [14]:
len(io_gpt[(io_gpt['outdated']==True) & (io_gpt['eval']==True)])

129

In [11]:
len(io_r1[io_llama['outdated']==True])

199

In [25]:
bg_gpt = bg_gpt.merge(bg_llama[['bankLabel', 'outdated']], on = 'bankLabel', how = 'left')

In [26]:
len(bg_gpt[(bg_gpt['outdated']==True) & (bg_gpt['eval']==True)])

12

In [28]:
len(bg_llama[bg_llama['outdated']==True])

12

In [43]:
len(io_r1)

325

In [44]:
len(io_r1[(io_r1['outdated']==True) & (io_r1['eval']==True)])

125

In [47]:
io_gpt = io_gpt.merge(io_r1[['organizationLabel', 'outdated']], on='organizationLabel', how='left')
io_gpt

,organizationLabel,officeholderLabel,confidence,answer,eval,outdated
0,African Development Bank,Akinwumi Adesina,high,Akinwumi Adesina,True,False
1,African Paralympic Committee,Leonel da Rocha Pinto,High,Samson Deen,True,True
2,Afro-Asian People's Solidarity Organization,Anwar Sadat,High,Dr. Helmy El-Hadidi,True,True
3,Aga Khan Development Network,Ismaili imam,high,His Highness Prince Rahim Aga Khan V,True,True
4,All European Academies,Antonio Loprieno,high,Paweł Rowiński,False,True
...,...,...,...,...,...,...
319,World Women’s Snooker,Mandy Fisher,high,Mandy Fisher,True,False
320,World Zionist Organization,Yaakov Hagoel,High,Tova Dorfman,True,True
321,World Baseball Softball Confederation,Riccardo Fraccari,high,Riccardo Fraccari,True,False
322,Young Christian Workers,Laetitia M. Navarro,High,Sarah Prenger,True,True


In [48]:
len(io_gpt[(io_gpt['outdated']==True) & (io_gpt['eval']==True)])

129

In [5]:
import pandas as pd

bg = pd.read_csv("bg_gpt_no_web.csv")
ceo = pd.read_csv("ceo_gpt_no_web.csv")
io = pd.read_csv("io_gpt_no_web.csv")

In [2]:
bg['eval'].mean()

0.6086956521739131

In [4]:
io['eval'].mean()

0.2153846153846154

In [6]:
ceo['eval'].mean()

0.47928994082840237

In [9]:
bg_gpt_web=pd.read_csv("bank_governors_01-04_gpt_web.csv")

In [10]:
from entity_align import extract_answer
bg_gpt_web['answer'] = bg_gpt_web['prediction'].apply(lambda x: extract_answer(x))
bg_gpt_web['confidence'] = bg_gpt_web['prediction'].str.extract(r'(?i)\b(high|medium|low)\b')

/root/autovu/venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [13]:
bg_gpt_web['eval'] = [
    rowwise_eval(a1, a2) if e1 is True else None
    for a1, a2, e1 in zip(ref_bg['answer'], bg_gpt_web['answer'], ref_bg['eval'])
]


In [14]:
bg_gpt_web.to_csv("bank_governors_01-04_gpt_web.csv", index=False)

In [15]:
bg_gpt_web = pd.read_csv("bank_governors_01-04_gpt_web.csv")


bg_gpt_web['eval'].mean()

1.0

In [2]:
from autovu import search

search_result = search.results("Who is the current CEO of LISI?")
formatted_results = "\n".join(
        f"Title: {res.get('title', 'N/A')}\nSnippet: {res.get('snippet', 'N/A')}\nDate: {res.get('date', 'N/A')}\n"
        for res in search_result['organic']
    )
print(formatted_results)

Title: Mike Reyes - LISI AEROSPACE NORTH AMERICA - LinkedIn
Snippet: Mike Reyes. Chief Executive Officer at LISI AEROSPACE NORTH AMERICA. LISI AEROSPACE NORTH AMERICA University of Phoenix ...
Date: N/A

Title: Governance - LISI GROUP
Snippet: Lionel RIVET Chief Executive Officer ; Marie-Georges BOUICHET Chief Quality & Regulatory Affairs Officer ; Kenneth DURHAM VP General Manager LISI MEDICAL US.
Date: N/A

Title: LISI AEROSPACE NORTH AMERICA | 28 comments - LinkedIn
Snippet: We are happy to announce the appointment of our new Chief Executive Officer - North America, Scott Wood, who will lead the Fasteners North ...
Date: Sep 6, 2024

Title: MANAGEMENT - LISI - Lisi Automotive
Snippet: François LIOTARD, Chief Executive Officer. Françoise ZAUGG, Finance, Marketing & Internal Control Director ; Martin BELEY, Managing Director Mechanical Safety ...
Date: N/A

Title: LISI Group CEO and Key Executive Team - Craft.co
Snippet: LISI Group's Chief Executive Officer, Director is Emmanuel Viell

In [4]:
with open("example.txt", "w", encoding="utf-8") as file:
    file.write(formatted_results)